In [31]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.image as img
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Raw Data Loading
train = pd.read_csv('../input/fashionmnist/fashion-mnist_train.csv')
test = pd.read_csv('../input/fashionmnist/fashion-mnist_test.csv')

display(train.head()); display(test.head())
display(train.info()); display(test.info())

# image 확인
figure = plt.figure()
ax = list()

img = train.drop('label', axis=1, inplace=False).values

for i in range(10):
  ax.append(figure.add_subplot(2,5,i+1))
  ax[i].imshow(img[i].reshape(28,28),
               cmap='Greys',
               interpolation='nearest')

plt.tight_layout()
plt.show()

# 결측치 없음
# print(train.isnull().sum().unique()); print(test.isnull().sum().unique()) # none

In [33]:
# combining train and test dataset
df = pd.concat([train, test], axis=0)
# print(df.shape) # (70000, 785)

# Data Split
train_x_data, test_x_data, train_t_data, test_t_data = \
train_test_split(df.drop('label', axis=1, inplace=False),
                 df['label'],
                 test_size=0.3,
                 random_state=1,
                 stratify=df['label'])

# # Data Split
# train_x_data, test_x_data, train_t_data, test_t_data = \
# train_test_split(train.drop('label', axis=1, inplace=False),
#                  train['label'],
#                  test_size=0.3,
#                  random_state=1,
#                  stratify=train['label'])

# x_data 정규화
scaler = MinMaxScaler()
scaler.fit(train_x_data)
norm_train_x_data = scaler.transform(train_x_data)
norm_test_x_data = scaler.transform(test_x_data)

In [34]:
# Keras
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3,
                 activation='relu',
                 input_shape=(28,28,1),
                 padding='valid',
                 strides=1))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3,
                 activation='relu',
                 padding='valid',
                 strides=1))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters=64, kernel_size=3,
                 activation='relu',
                 padding='valid',
                 strides=1))

model.add(Flatten())
model.add(Dropout(rate=0.5))
model.add(Dense(units=256, # Hidden Layer
                activation='relu'))

model.add(Dense(units=10, # Output Layer
                activation='softmax'))

print(model.summary())

In [35]:
model.compile(optimizer=Adam(learning_rate=1e-3),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
# Check Point
checkpoint_path = '../kaggle/working/Fashion_MNIST/cp-{epoch:04d}.ckpt'
cp_callback = ModelCheckpoint(checkpoint_path,
                              save_weights_only=True,
                              period=5,
                              verbose=1)

# Early Stopping
es = EarlyStopping(monitor='val_loss',
                   min_delta=0.001,
                   patience=5,
                   verbose=1,
                   mode='auto',
                   restore_best_weights=True)

history = model.fit(norm_train_x_data.reshape(-1,28,28,1),
                    train_t_data,
                    epochs=200,
                    batch_size=100,
                    verbose=1,
                    validation_split=0.3,
                    callbacks=[cp_callback, es])

model.save('../kaggle/working/Fashion_MNIST/MNIST_Model_save.h5')

In [37]:
figure = plt.figure(figsize=(15, 5))

ax1 = figure.add_subplot(1,2,1)
ax2 = figure.add_subplot(1,2,2)

ax1.set_title('Loss Trend')
ax1.set_xlabel('epoch')
ax1.set_ylabel('loss')

ax1.plot(history.history['loss'], color='r')
ax1.plot(history.history['val_loss'], color='b')
ax1.grid()

ax2.set_title('Accuracy Trend')
ax2.set_xlabel('epoch')
ax2.set_ylabel('accuracy')

ax2.plot(history.history['accuracy'], color='r')
ax2.plot(history.history['val_accuracy'], color='b')
ax2.grid()

ax1.legend(['train', 'validation'], loc='upper right')
ax2.legend(['train', 'validation'], loc='lower right')
plt.show()

# epoch는 10번 정도가 적당하다

In [40]:
# Loading Model
new_model = load_model('../kaggle/working/Fashion_MNIST/MNIST_Model_save.h5')

# Evaluation
# model.evaluate(norm_test_x_data.reshape(-1,28,28,1), test_t_data) # loss: 0.2410 - accuracy: 0.9135

new_model.evaluate(norm_test_x_data.reshape(-1,28,28,1), test_t_data) # loss: 0.2410 - accuracy: 0.9135